In [1]:
import os
import glob
import datetime
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('max_rows', 300)
pd.set_option('max_columns', 200)

perc = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]

startDate = '20201228'
endDate = '20210108'

readPath = r'L:\orderLog\data\***'
dataPathLs = np.array(glob.glob(readPath))
dateLs = np.array([os.path.basename(i).split('.')[0] for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
print(dataPathLs)

rawOrderLog = []
for thisPath in dataPathLs:
    data = pickle.load(open(thisPath, 'rb'))
    data = data.rename(columns={'mdClockAtArrival': 'caamd'})
    rawOrderLog += [data]
rawOrderLog = pd.concat(rawOrderLog, sort=False)


for col in ['clockAtArrival', 'caamd', 'secid', 'updateType', 'vai', 'absFilledThisUpdate', 'orderDirection', 'absOrderSize',
            'absOrderSizeCumFilled', 'date', 'accCode', 'mse']:
    rawOrderLog[col] = rawOrderLog[col].fillna(0).astype('int64')   
rawOrderLog = rawOrderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

rawOrderLog = rawOrderLog[rawOrderLog["secid"] >= 1000000]

if rawOrderLog[rawOrderLog.duplicated(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep=False)].shape[0] != 0:
    display('There are accounts with duplicated ticks:')
    display(rawOrderLog[rawOrderLog.duplicated(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep=False)]\
.groupby(['date', 'colo', 'accCode'])['ars'].size())
    rawOrderLog = rawOrderLog.drop_duplicates(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep='first')

display('There are ticks with orderDirection 0')
display(rawOrderLog[rawOrderLog['orderDirection'] == 0][['date', 'colo', 'accCode', \
            'secid', 'vai', 'updateType', 'sdd', 'orderDirection', 'absOrderSize', 'internalId', 'orderId']])

assert(rawOrderLog[rawOrderLog['updateType'] == 0][rawOrderLog[rawOrderLog['updateType'] == 0]\
                                                   .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                'vai', 'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
try:
    assert(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
except:
    print('There are orders with all things same except sdd')
    print(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'absOrderSize', 'internalId'], keep=False)])
    assert(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'absOrderSize', 'internalId', 'sdd'], keep=False)].shape[0] == 0)
try:
    assert(sum(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() != 1) == 0) 
except:
    print('There are orders with same internalId but different orderId other than accCode 8856 case')
    print(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique()[rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() > 1])

r2 = rawOrderLog[(rawOrderLog['accCode'] != 8856) & (rawOrderLog['orderDirection'] != 0)]
r1 = rawOrderLog[(rawOrderLog['accCode'] == 8856) & (rawOrderLog['orderDirection'] != 0)]
r1['test'] = r1.groupby(['date', 'colo', 'accCode', 'secid', 
            'orderDirection', 'absOrderSize']).grouper.group_info[0]
r1 = r1.sort_values(by=['test', 'clockAtArrival'])
r1.loc[r1['updateType'] != 0, 'vai'] = np.nan
r1['vai'] = r1.groupby('test')['vai'].ffill()
r2['test'] = r2.groupby(['date', 'colo', 'accCode', 'secid', 
            'orderDirection', 'absOrderSize', 'internalId']).grouper.group_info[0]
r2 = r2.sort_values(by=['test', 'clockAtArrival'])
r2.loc[r2['updateType'] != 0, 'vai'] = np.nan
r2['vai'] = r2.groupby('test')['vai'].ffill()
try:
    assert(sum(r1[r1['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1) == 0)
except:
    a = r1[r1['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique()[r1[r1['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1].reset_index()
    print('There are orders in 8856 with same internalId and various orderId!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    print(pd.merge(r1, a[['test', 'vai']], on=['test', 'vai'], how='inner')[['secid', 'accCode', 'colo', 'vai', 'updateType', 'sdd', 'internalId', 'orderId', 'absOrderSize', 'absFilledThisUpdate', 'absOrderSizeCumFilled', 'orderPrice', 'tradePrice']])

try:
    assert(sum(r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1) == 0)
except:
    a = r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique()[r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1].reset_index()
    print('There are orders out of 8856 with same internalId and various orderId!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    print(pd.merge(r2, a[['test', 'vai']], on=['test', 'vai'], how='inner')[['secid', 'accCode', 'colo', 'vai', 'updateType', 'sdd', 'internalId', 'orderId', 'absOrderSize', 'absFilledThisUpdate', 'absOrderSizeCumFilled', 'orderPrice', 'tradePrice']])
orderLog = pd.concat([r1, r2])
del r1
del r2    

orderLog = orderLog.sort_values(by=['date', 'colo', 'accCode', 'secid', 'vai', 'clockAtArrival']).reset_index(drop=True)
orderLog['clock'] = orderLog['clockAtArrival'].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
orderLog["broker"] = np.where(orderLog["accCode"].astype(str).apply(lambda x: len(x) == 6), orderLog['accCode'] // 10000, orderLog['accCode'] // 100)
orderLog['colo_broker'] = orderLog['colo'].str[:2] + '_' + orderLog['broker'].astype('str')
orderLog['order'] = orderLog.groupby(['date', 'colo', 'accCode', 'secid', 'vai', 'orderDirection', 'absOrderSize', 'internalId']).grouper.group_info[0]
orderLog['group'] = orderLog.groupby(['date', 'secid', 'vai']).grouper.group_info[0]
orderLog['startClock'] = orderLog.groupby(['order'])['clockAtArrival'].transform('first')
orderLog['duration'] = orderLog['clockAtArrival'] - orderLog['startClock']
orderLog['orderPrice'] = orderLog['orderPrice'].apply(lambda x: round(x, 2))
orderLog['tradePrice'] = orderLog['tradePrice'].apply(lambda x: round(x, 2))
orderLog['orderDirection1'] = np.where(orderLog["orderDirection"] == -2, -1, np.where(
    orderLog["orderDirection"] == 2, 1, orderLog["orderDirection"]))
orderLog["ars"] = orderLog.groupby(['order'])['ars'].transform('first')
orderLog['sdd'] = orderLog.groupby('order')['sdd'].transform('first')
orderLog['caamd'] = orderLog.groupby('order')['caamd'].transform('first')
df = orderLog.copy()


### Assertion 1:  make sure same direction in same date, secid, vai
print('=======================================================================================')
print('1. same date, secid, vai: same direction')
taking = orderLog[orderLog['ars'].isin([1, 7])]
making = orderLog[orderLog['ars'].isin([2, 3])]
else_orders = orderLog[~(orderLog['ars'].isin([1, 2, 3, 7]))]
display('orders with abnormal ars values')
display(else_orders[else_orders['updateType'] == 0].groupby('ars')['date'].size().sort_values(ascending=False))
display(else_orders[else_orders['updateType'] == 0].groupby('accCode')['date'].size().sort_values(ascending=False))
# else_orders[else_orders['updateType'] == 0].groupby('ars')['date'].size().sort_values(ascending=False).reset_index()['ars'].values
# kk = else_orders[else_orders['updateType'] == 0].groupby('accCode')['date'].size().sort_values(ascending=False)
# kk[kk > 20].index.values
taking['directNum'] = taking.groupby(['date', 'secid', 'vai', 'sdd'])['orderDirection1'].transform('nunique')

if len(taking[taking['directNum'] != 1]) > 0:
    print('opposite direction for same date, same secid, same vai')
    display(taking[(taking['directNum'] != 1) & (taking['updateType'] == 0)].groupby(['accCode'])['orderDirection'].size())
    taking = taking[taking['directNum'] == 1]

assert((taking.groupby(['date', 'secid', 'vai', 'sdd'])['orderDirection1'].nunique() == 1).all() == True)
orderLog = pd.concat([taking, making]).sort_values(by=['date', 'colo', 'accCode', 'secid', 'vai', 'clockAtArrival']).reset_index(drop=True)

## Assertion 2:  make sure each account, secid, vai only has one insertion
print('=======================================================================================')
print('2. same date, secid, vai, accCode: one insertion')
a = orderLog[orderLog['updateType'] == 0].groupby(['date', 'colo', 'accCode', 'secid', 'vai', 'sdd'])['clockAtArrival'].count()
if len(a[a > 1]) > 0:
    print('more than one insertion at same time')
    a = a[a>1].reset_index()
    display(a)
    display(a.groupby(['accCode'])['date'].size())      
    d_el = pd.merge(orderLog, a, on=['date', 'colo', 'accCode', 'secid', 'vai', 'sdd'])['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(d_el))]

orderLog['isMsg'] = np.where(orderLog['updateType'] == 0, 
                             np.where(orderLog['mse'] == 100, 1, 0), np.nan)
orderLog['isMsg'] = orderLog.groupby(['order'])['isMsg'].ffill()

placeSZE = orderLog[(orderLog['secid'] >= 2000000) & (orderLog['updateType'] == 0)]
print('%.2f%% SZE orders triggered by msg data'%(placeSZE[placeSZE['isMsg'] == 1].shape[0]/placeSZE.shape[0]*100))


### Assertion 3:  check IPO stocks selling status
print('=======================================================================================')
print('3. IPO stocks selling (ars = 301, 302)')
if orderLog[orderLog['ars'].isin([301, 302])].shape[0] != 0:
    kk = orderLog[orderLog['ars'].isin([301, 302])]
    print(kk)
    try:
        assert(kk[kk['orderDirection1'] == 1].shape[0] == 0)
        print('we only sell, never buy')
    except:
        print('There are IPO buy side orders!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(kk[kk['orderDirection1'] == 1])
    kk1 = kk[kk['updateType'] == 0]
    kk1 = kk1.sort_values(by=['accCode', 'secid','clockAtArrival'])
    kk1['diff'] = kk1.groupby(['accCode', 'secid'])['clockAtArrival'].apply(lambda x: x-x.shift(1))
    kk1['diff'] = kk1['diff'].fillna(0)
    try:
        assert(kk1[kk1['diff'] < 10e6].shape[0] == 0)
        print('for each stock in the same account, there is no insertion within 10 seconds of the previous insertion')
    except:
        print('There are insertion within 10 seconds for orders under same account same stock!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(kk1[kk1['diff'] < 10e6])
    kk2 = kk[(kk['updateType'] == 1)]
    try:
        assert(kk2[kk2['duration'] < 3e6].shape[0] == 0)
        print('for each stock in the same account, the cancellation of an order happens more than 3 seconds after the insertion')
    except:
        print('There are cancellation within 3 seconds for orders under same account same stock!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(kk2[kk2['duration'] < 3e6])


### Assertion 4: check updateType == 7 orders, make sure updateType == 7 orders < 20 per account, < 100 in total
print('=======================================================================================')
print('4. updateType 7 orders')
if orderLog[orderLog['updateType'] == 7].shape[0] != 0:
    assert(orderLog[orderLog['updateType'] == 7].groupby(['date', 'accCode'])['order'].nunique().max() < 20)
    assert(orderLog[orderLog['updateType'] == 7].groupby(['date', 'accCode'])['order'].nunique().sum() < 100)

### Assertion 5: check updateType == 6 orders, make sure updateType == 6 orders < 5% per account
print('=======================================================================================')
print('5. updateType 6 orders')
k1 = orderLog[orderLog['updateType'] == 6].groupby(['date', 'accCode'])['order'].nunique().reset_index()
k2 = orderLog.groupby(['date', 'accCode'])['order'].nunique().reset_index()
k = pd.merge(k1, k2, on=['date', 'accCode'], how='left')
k['prob'] = k['order_x']/k['order_y']
try:
    assert(sum(k['prob'] >= 0.05) == 0)
except:
    print('There are accounts with more than 5% updateType 6 orders!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    print(k[k['prob'] >= 0.05])

### Assertion 6: check CYB orders, make sure all CYB stocks have absOrderSize < 30w
print('=======================================================================================')
print('6. CYB stocks order size < 30w')
try:
    cyb = orderLog[(orderLog['secid'] >= 2300000) & (orderLog['updateType'] == 0)].groupby('date')['absOrderSize'].max().reset_index()
    assert(cyb[cyb['absOrderSize'] > 300000].shape[0] == 0)
except:
    print('CYB stocks total absOrderSize >= 30w!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    
     
### Assertion 7:  make sure there is no unexpected updateType 
print('=======================================================================================')
print('7. unexpected updateType')
def getTuple(x):
    return tuple(i for i in x)

checkLog = orderLog[~((orderLog['updateType'] == 4) & (orderLog.groupby(['order'])['updateType'].shift(-1) == 4))]
checkLog = checkLog.groupby(['order'])['updateType'].apply(lambda x: getTuple(x)).reset_index()
checkLog['status'] = np.where(checkLog['updateType'].isin([(0, 2, 4), (0, 2, 1, 4), (0, 2, 1, 2, 4), (0, 2, 4, 1, 4), (0, 4), (0, 1, 4), (0, 4, 1, 4), (0, 2, 2, 4), (0, 4, 2, 4), (0, 2, 2, 1, 4), (0, 2, 2, 4, 1, 4)]),0,
                     np.where(checkLog['updateType'].isin([(0, 2, 4, 1, 3), (0, 2, 4, 1, 4, 3), (0, 2, 1, 4, 3), (0, 4, 1, 3), (0, 1, 4, 3),
                                                               (0, 2, 2, 4, 1, 3), (0, 2, 2, 4, 1, 4, 3), (0, 2, 2, 1, 4, 3), (0, 4, 2, 4, 1, 3),
                                                               (0, 4, 2, 1, 3), (0, 4, 1, 4, 3), (0, 4, 1)]), 1,
                     np.where(checkLog['updateType'].isin([(0, 2, 1, 3), (0, 2, 2, 1, 3), (0, 2, 3), (0, 3), (0, 1, 3), (0, ), (0, 2), (0, 2, 1), (0, 2, 2)]), 2, 3)))
display(checkLog[checkLog['status'] == 3].groupby('updateType')['order'].size())
orderLog = pd.merge(orderLog, checkLog[['order', 'status']], how='left', on=['order'], validate='many_to_one')
orderLog = orderLog[orderLog['status'].isin([0, 1, 2])].reset_index(drop=True)

### Assertion 8:  make sure status==0 got all traded
print('=======================================================================================')
print('8. status == 0: all traded')
a = orderLog[orderLog['status'] == 0]
a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
a.columns = ['order', 'filled', 'total']
print('in total trade, any fill != total cases')
display(a[a['filled'] != a['total']])
if a[a['filled'] != a['total']].shape[0] > 0:
    removeOrderLs = a[a['filled'] != a['total']]['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]
    
### Assertion 9:  make sure status==1 got partial traded
print('=======================================================================================')
print('9. status == 1: partial traded')
a = orderLog[orderLog['status'] == 1]
a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
a.columns = ['order', 'filled', 'total']
print('in partial trade, any fill >= total or fill is 0 cases for updateType 4')
display(a[(a['filled'] >= a['total']) | (a['filled'] == 0)])
if a[(a['filled'] >= a['total']) | (a['filled'] == 0)].shape[0] > 0:
    removeOrderLs = a[(a['filled'] >= a['total']) | (a['filled'] == 0)]['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]
    
### Assertion 10: make sure no cancellation within 1 sec
print('=======================================================================================')
print('10. no cancellation within 1 sec')
a = orderLog[(orderLog['updateType'] == 1) & (orderLog['duration'] < 1e6)]
print('any cancellation within 1 sec')
display(a)
if a.shape[0] > 0:
    removeOrderLs = a['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]


### Assertion 11: make sure no order has shares > 80w or notional > 800w
print('=======================================================================================')
print('11. Orders with size > 80w or notional > 800w')
orderLog['orderNtl'] = orderLog['absOrderSize'] * orderLog['orderPrice']
if orderLog[orderLog['absOrderSize'] > 800000].shape[0] > 0:
    print('some order quantity are > 80w')
    print(orderLog[orderLog['absOrderSize'] > 800000].groupby(['colo', 'accCode'])['order'].nunique())
    display(orderLog[orderLog['absOrderSize'] > 800000][['date', 'accCode', 'secid', 'vai', 'absOrderSize', 'orderPrice',
                                                         'orderNtl', 'orderDirection', 'clock', 'order']])
            
if orderLog[orderLog['orderNtl'] > 8000000].shape[0] > 0:
    print('some order ntl are > 800w')
    print(orderLog[orderLog['orderNtl'] > 8000000].groupby(['colo', 'accCode'])['order'].nunique())
    display(orderLog[orderLog['orderNtl'] > 8000000][['date', 'accCode', 'secid', 'vai', 'absOrderSize', 'orderPrice',
                                                      'orderNtl', 'orderDirection', 'clock', 'order', "updateType", 
                                                      "tradePrice", "absOrderSizeCumFilled", "absFilledThisUpdate"]])

removeOrderLs = list(set(orderLog[orderLog['absOrderSize'] > 800000]['order'].unique()) | set(orderLog[orderLog['orderNtl'] > 8000000]['order'].unique()))
orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]


orderLog = orderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

orderLog["mrsb90"] = orderLog.groupby(['order'])['mrsb90'].transform('first')
orderLog["mrss90"] = orderLog.groupby(['order'])['mrss90'].transform('first')
orderLog["aaa"] = orderLog.groupby(['order'])['aaa'].transform('first')

orderLog['m1'] = orderLog['mrstaat'].apply(lambda x: x - (x // 10000) * 10000)
orderLog['m2'] = orderLog['mrstauc'].apply(lambda x: x - (x // 10000) * 10000)
try:
    orderLog['mrsb90'] = orderLog['mrsb90'].astype(float)
except:
    print(orderLog[orderLog['mrsb90'] == '-'])
    orderLog = orderLog[orderLog['mrsb90'] != '-']
    orderLog['mrsb90'] = orderLog['mrsb90'].astype(float)
try:
    orderLog['mrss90'] = orderLog['mrss90'].astype(float)
except:
    print(orderLog[orderLog['mrss90'] == '-'])
    orderLog = orderLog[orderLog['mrss90'] != '-']
    orderLog['mrss90'] = orderLog['mrss90'].astype(float)
try:
    orderLog['aaa'] = orderLog['aaa'].astype(float)
except:
    print(orderLog[orderLog['aaa'] == '-'])
    orderLog = orderLog[orderLog['aaa'] != '-']
    orderLog['aaa'] = orderLog['aaa'].astype(float)
orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'mrstauc'] = \
orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'm2']

orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'mrstaat'] = \
orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'm1']

orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'mrstauc'] = \
orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'm2']

orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'mrstaat'] = \
orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'm1']    

orderLog['exchange'] = np.where(orderLog['secid'] >= 2000000, 'SZE', 'SSE')
orderLog['orderNtl'] = orderLog['orderPrice'] * orderLog['absOrderSize']
orderLog['tradeNtl'] = np.where(orderLog['updateType'] == 4, orderLog['tradePrice']*orderLog['absFilledThisUpdate'], 0)
orderLog["mrstaat"] = orderLog.groupby(['order'])['mrstaat'].transform('first')
orderLog["mrstauc"] = orderLog.groupby(['order'])['mrstauc'].transform('first')
orderLog['sta'] = np.where(orderLog['mrstaat'] == 1000, '1. staone', np.where(
orderLog['mrstaat'] == 3000, '2. statwo', np.where(
orderLog['mrstaat'].isin([11000, 13000]), '3. sta300', 'else')))
display(orderLog[orderLog['sta'] == 'else'][['date', 'secid', 'colo', 'accCode', 'updateType', 'vai', 'absOrderSize', 'orderDirection', 'mrstaat', 'mrstauc', 'sdd', 'ApplSeqNum', 'sta']])
orderLog = orderLog[orderLog['mrstaat'].isin([11000, 13000, 1000, 3000])]
display(orderLog[orderLog['updateType'] == 0].groupby(['mrstaat', 'mrstauc'])['date'].size())

['L:\\orderLog\\data\\20201228.pkl' 'L:\\orderLog\\data\\20201231.pkl'
 'L:\\orderLog\\data\\20201229.pkl' 'L:\\orderLog\\data\\20201230.pkl'
 'L:\\orderLog\\data\\20210104.pkl' 'L:\\orderLog\\data\\20210105.pkl'
 'L:\\orderLog\\data\\20210106.pkl' 'L:\\orderLog\\data\\20210107.pkl'
 'L:\\orderLog\\data\\20210108.pkl']


'There are ticks with orderDirection 0'

,date,colo,accCode,secid,vai,updateType,sdd,orderDirection,absOrderSize,internalId,orderId
314035,20201228,zs_66_01,6634,2000007,-1,1,49597.0,0,0,-1.0,1.812811e+10
342328,20201228,zs_52_09,5291,2000553,-1,1,53524.0,0,0,-1.0,1.854590e+06
347596,20201228,zs_54_01,5474,2000608,-1,1,47253.0,0,0,-1.0,8.689127e+08
349238,20201228,zs_54_01,5474,2000619,-1,7,47362.0,0,0,-1.0,8.689128e+08
423299,20201228,zs_96_08,9685,2002100,-1,1,48345.0,0,0,-1.0,7.221449e+17
424530,20201228,zs_92_02,9243,2002108,-1,1,40760.0,0,0,-1.0,8.250502e+09
456478,20201228,zs_96_08,9685,2002274,-1,1,53632.0,0,0,-1.0,7.221449e+17
483038,20201228,zs_52_09,5264,2002410,-1,1,34355.0,0,0,-1.0,9.338200e+04
517378,20201228,zs_54_01,5474,2002598,-1,1,53196.0,0,0,-1.0,8.689184e+08
554629,20201228,zs_54_01,5474,2002765,-1,1,53781.0,0,0,-1.0,8.689190e+08


There are orders with all things same except sdd
         ApplSeqNum       aaa  absFilledThisUpdate  absOrderSize  \
1361311   1444460.0  0.000687                    0           100   
1361371   9795909.0  0.000694                    0           100   

         absOrderSizeCumFilled  accCode  ars             caamd  \
1361311                      0   966301  1.0  1609205575014635   
1361371                      0   966301  1.0  1609208372125359   

         cancellationPending        cfe         clock    clockAtArrival  \
1361311                  0.0  128034.25  1.609206e+15  1609205575014666   
1361371                  0.0   52600.79  1.609208e+15  1609208372125429   

         cumSharesBought  cumSharesBuyInserted  cumSharesSellInserted  \
1361311              0.0                   0.0                  300.0   
1361371              0.0                   0.0                  600.0   

         cumSharesSold      date  finalState        gfe       hee  \
1361311          200.0  20201229

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


...

10. no cancellation within 1 sec
any cancellation within 1 sec


,ApplSeqNum,aaa,absFilledThisUpdate,absOrderSize,absOrderSizeCumFilled,accCode,ars,caamd,cancellationPending,cfe,clock,clockAtArrival,cumSharesBought,cumSharesBuyInserted,cumSharesSellInserted,cumSharesSold,date,finalState,gfe,hee,insertedShortOrder,insertionPending,internalId,inv_L,inv_L0,inv_S,inv_S0,l4algoDebug,l4tr,locateShares,locateSharesTotal,mfe,mra100,mrb100,mrm,mrm25,mrmum,mrrlma,mrsb300,mrsb90,mrss300,mrss90,mrstaat,mrstauc,mrstaum,mrv,ms,mse,mt,mta,mv,orderDirection,orderId,orderOutstanding,orderPrice,orderSysId,resa,sdd,secid,sequenceNo,session,threadId,totalActions,totalCanceled,tradeId,tradePrice,underlyingIndex,updateType,vai,zipFile,colo,caa_orderLog,start_time,Price,OrderQty,Side,statusLs,TradePriceLs,TradeQtyLs,beta_60,adjMid_F30s,adjMid_F90s,adjMid_F300s,indexClose,indexClose_F30s,indexClose_F90s,indexClose_F300s,test,broker,colo_broker,order,group,startClock,duration,orderDirection1,directNum,isMsg,status
2262518,0.0,-1,0,100,0,8854,1.0,1609307843130063,1.0,5423341.79,2020-12-30 13:57:23.309056,1609307843309056,0.0,0.0,100.0,0.0,20201230,0.0,5423340.79,-1.0,0.0,0.0,190.0,100.0,0.0,0.0,0.0,1345.000000|,0.0,0.0,0.0,-1.0,1957.0,1951.0,-0.004078,0.000603,-1.0,-1.0,-0.003175,-0.001534,0.000110,-0.001057,13000.0,11000.0,-1.0,17761035.0,13:57:23.307447,22,-1.0,-1,0.0,-1,8.885000e+03,1.0,19.58,0051008885,2.0,135722000.0,1603008,10463942.0,2,122924.0,571.0,42.0,NaN,-1.0,852,1,17760235.0,logs_20201230_zt_88_06_day_8854,zt_88_06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.823186,19.564615,19.568571,19.830833,6527.2886,6527.3770,6527.7307,6532.0858,567341,88,zt_88,567878,282055,1609307843130266,178790,-1,1.0,0.0,2
2262630,0.0,-1,0,100,0,8854,1.0,1609298373762077,1.0,4339395.87,2020-12-30 11:19:34.180024,1609298374180024,0.0,0.0,200.0,100.0,20201230,0.0,4339394.87,-1.0,0.0,0.0,287.0,6700.0,0.0,0.0,0.0,1115.000000|,0.0,0.0,0.0,-1.0,3494.0,3475.0,-0.000817,0.002538,-1.0,-1.0,-0.002911,-0.003110,-0.002534,-0.002361,13000.0,11000.0,-1.0,3483963.0,11:19:34.178594,22,-1.0,-1,0.0,-1,6.477000e+03,1.0,34.96,0051006477,2.0,111933000.0,1603456,4386801.0,3,114097.0,307.0,20.0,NaN,-1.0,852,1,3483663.0,logs_20201230_zt_88_06_day_8854,zt_88_06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.138497,34.924545,35.016154,34.854000,6534.0868,6535.3699,6538.0923,6536.0611,567378,88,zt_88,567913,287957,1609298373762174,417850,-1,1.0,0.0,2
2262661,0.0,-1,0,200,0,8854,1.0,1609296809192182,1.0,4226087.47,2020-12-30 10:53:29.509160,1609296809509160,600.0,800.0,0.0,0.0,20201230,0.0,4226086.47,-1.0,0.0,0.0,201.0,600.0,600.0,0.0,0.0,1045.000000|,0.0,0.0,0.0,-1.0,7262.0,7261.0,0.001725,0.004244,-1.0,-1.0,0.001422,0.000137,-0.001558,-0.000264,13000.0,11000.0,-1.0,1703499.0,10:53:29.507618,22,-1.0,-1,0.0,1,5.459000e+03,1.0,72.54,0051005459,2.0,105328000.0,1603583,2334346.0,3,114104.0,213.0,12.0,NaN,-1.0,852,1,1703299.0,logs_20201230_zt_88_06_day_8854,zt_88_06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.583953,72.606667,72.725000,73.204615,6526.7336,6528.0027,6531.0701,6533.7960,567387,88,zt_88,567921,288887,1609296809192395,316765,1,1.0,0.0,2
2262732,0.0,-1,0,100,0,8854,1.0,1609306975678291,1.0,5409570.09,2020-12-30 13:42:56.338860,1609306976338860,0.0,0.0,100.0,0.0,20201230,0.0,5409569.09,-1.0,0.0,0.0,152.0,4250.0,0.0,0.0,0.0,1330.000000|,0.0,0.0,0.0,-1.0,6699.0,6637.0,0.002238,-6.9e-05,-1.0,-1.0,-0.004424,-0.006091,-0.004853,-0.002701,13000.0,11000.0,-1.0,6335649.0,13:42:56.337460,22,-1.0,-1,0.0,-1,8.476000e+03,1.0,67.00,0051008476,2.0,134254000.0,1603678,9310481.0,2,122930.0,530.0,39.0,NaN,-1.0,852,1,6317849.0,logs_20201230_zt_88_06_day_8854,zt_88_06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.003676,66.483333,66.538750,66.908438,6523.0712,6522.6492,6523.4124,6526.4314,567405,88,zt_88,567942,290330,1609306975678489,660371,-1,1.0,0.0,2
2262803,0.0,-1,0,100,0,8854,1.0,1609307131162699,1.0,5418525.78,2020-12-30 13:45:31.933621,1609307131933621,2800.0,5600.0,0.0,0.0,20201230,0.0,5418524.78,-1.0,0.0,0.0,170.0,5710.0,5700.0,0.0,0.0,1345.000000|,0.0,0.0,0.0,-1.0,1559.0,1552.0,0.000841,0.003929,-1.

11. Orders with size > 80w or notional > 800w


,date,secid,colo,accCode,updateType,vai,absOrderSize,orderDirection,mrstaat,mrstauc,sdd,ApplSeqNum,sta
19860,20201228,1600141,zt_68_02,687801,0,38535.0,200,-1,NaN,NaN,145635000.0,NaN,else
228906,20201228,1603377,zt_88_03,897002,0,3837667.0,900,1,NaN,NaN,145216000.0,NaN,else
568807,20201228,2002830,zs_52_06,5269,0,1.0,2000,-1,NaN,NaN,145645010.0,NaN,else
1491409,20201229,2300207,zs_94_03,9448,0,28688798.0,300,-1,NaN,NaN,145442190.0,NaN,else
1775959,20201230,1600740,zt_52_07,527101,0,33348760.0,900,1,NaN,NaN,145646000.0,NaN,else
1804978,20201230,1600987,zt_94_06,9551,0,3600403.0,8700,-1,NaN,NaN,145635000.0,NaN,else
1899507,20201230,1603777,zt_96_09,968501,0,1025001.0,5300,-1,2000.0,1000.0,133619000.0,0.0,else
1899508,20201230,1603777,zt_96_09,968501,2,1025001.0,5300,-1,2000.0,1000.0,133619000.0,0.0,else
1899509,20201230,1603777,zt_96_09,968501,4,1025001.0,5300,-1,2000.0,1000.0,133619000.0,0.0,else
1899510,20201230,1603777,zt_96_09,968501,4,1025001.0,5300,-1,2000.0,1000.0,133619000.0,0.0,else


mrstaat  mrstauc
1000.0   0.0        260454
3000.0   1000.0     891509
         2000.0      71355
         3000.0      30048
11000.0  10000.0     23345
         20000.0      1993
         30000.0      1276
13000.0  11000.0    552674
         12000.0     17667
         13000.0      3303
         21000.0     16899
         22000.0     17136
         23000.0      3046
         31000.0      7918
         32000.0      6315
         33000.0      7508
Name: date, dtype: int64

In [8]:
orderLog[orderLog['accCode'] == 537401].groupby('date')['colo'].unique()

date
20201228    [zt_52_07]
20201229    [zt_52_07]
20201230    [zt_52_07]
20201231    [zt_52_07]
20210104    [zt_52_07]
20210107    [zt_52_12]
20210108    [zt_52_12]
Name: colo, dtype: object

In [2]:
orderLog[orderLog['colo'].str[:5].isin(['zs_52', 'zt_52']) & (orderLog['updateType'] == 0)].groupby(['date', 'colo'])['accCode'].unique()

date      colo    
20201228  zs_52_06    [5332, 5384, 5273, 5222, 5269, 5287, 5232, 5275]
          zs_52_08                      [5292, 5386, 5281, 5276, 5329]
          zs_52_09          [5264, 5225, 5326, 5230, 5290, 5291, 5289]
          zt_52_04            [522601, 522201, 528401, 526901, 537301]
          zt_52_05                    [537403, 527601, 522901, 528701]
          zt_52_07    [523001, 537401, 528101, 527701, 532801, 527101]
          zt_52_10    [528901, 522401, 523101, 529001, 529101, 522501]
20201229  zs_52_06    [5332, 5384, 5273, 5222, 5269, 5287, 5232, 5275]
          zs_52_08                [5292, 5377, 5386, 5281, 5329, 5276]
          zs_52_09          [5264, 5326, 5230, 5290, 5225, 5291, 5289]
          zt_52_04            [528401, 522601, 522201, 537301, 526901]
          zt_52_05                    [537403, 527601, 522901, 528701]
          zt_52_07    [537401, 523001, 527701, 528101, 532801, 527101]
          zt_52_10    [529101, 523101, 529001, 522401, 528

In [4]:
orderLog[orderLog['colo'].isin(['zs_52_13', 'zt_52_12'])].groupby(['date', 'exchange', 'colo'])['accCode'].unique()

date      exchange  colo    
20201231  SSE       zs_52_13                      [5386]
                    zt_52_12                    [537301]
          SZE       zs_52_13                      [5386]
20210104  SSE       zs_52_13                      [5386]
                    zt_52_12                    [537301]
          SZE       zs_52_13                      [5386]
20210105  SSE       zt_52_12            [537301, 537403]
20210106  SSE       zs_52_13                      [5386]
                    zt_52_12            [537301, 537403]
          SZE       zs_52_13                [5329, 5386]
20210107  SSE       zs_52_13                [5377, 5386]
                    zt_52_12    [537401, 537403, 537301]
          SZE       zs_52_13          [5329, 5377, 5386]
20210108  SSE       zs_52_13                [5377, 5386]
                    zt_52_12    [537401, 537301, 537403]
          SZE       zs_52_13          [5329, 5377, 5386]
Name: accCode, dtype: object

In [74]:
orderLog = orderLog[~orderLog['mrstauc'].isnull()]
orderLog['sta_uc'] = orderLog['sta'] + '_' + orderLog['mrstauc'].astype(int).astype(str)
orderLog["server"] = orderLog["colo"].apply(lambda x: x.split("_")[0] + x.split("_")[1] + x.split("_")[2])
orderLog["server_account"] = orderLog["server"] + '_' + orderLog['accCode'].astype('str')
checkLog = orderLog[orderLog['updateType'].isin([0, 3, 4])].reset_index(drop=True)
checkLog = checkLog[checkLog['duration'] <= 1e6]

checkLog["time"] = checkLog["clock"].apply(lambda x: x.strftime("%H%M%S")).astype(int)
# ol = checkLog[(checkLog["updateType"] == 0) & (~checkLog["ars"].isin([121, 221, 321, 131, 231, 331])) & (checkLog["time"]>= 93000)
#              & (checkLog["time"] < 93300)]["order"].unique()

sta_list = orderLog['sta_uc'].unique()

result = {}
for col in ['date', 'broker1', 'broker2', 'isFast']:
    result[col] = []
            
for i in sta_list:
    checkLog1 = checkLog[checkLog['sta_uc'] == i]
    checkLog1['accountNum'] = checkLog1.groupby(['date', 'secid', 'vai'])['server_account'].transform('nunique')
    checkLog1 = checkLog1[checkLog1['accountNum'] >= 2]
    checkLog1['cumFillSize'] = checkLog1.groupby(['order'])['absOrderSizeCumFilled'].transform('max')
    checkLog1['status'] = np.where(checkLog1['cumFillSize'] == 0, 2, 
                                    np.where(checkLog1['cumFillSize'] < checkLog1['absOrderSize'], 1, 0))
    checkLog1 = checkLog1.groupby(['group', 'order'])[['date', 'server_account', 'secid', 'vai', 'status']].first().reset_index()

    for exchg in ['SZE']:

        print(exchg)

        if exchg == 'SSE':
            exchangeLog = checkLog1[checkLog1['secid'] < 2000000]
        else:
            exchangeLog = checkLog1[checkLog1['secid'] >= 2000000]

        for group, groupData in exchangeLog.groupby(['group']):
            date = groupData['date'].values[0]
            brokerLs = groupData['server_account'].values
            if len(brokerLs) == 0:
                continue
            statusLs = groupData['status'].values
            ixLs = [i for i in range(len(brokerLs))]
            for k, broker1, status1 in zip(ixLs[:-1], brokerLs[:-1], statusLs[:-1]):
                for broker2, status2 in zip(brokerLs[k+1:], statusLs[k+1:]):
                    if broker1 != broker2:
                        result['date'] += [date, date]
                        result['broker1'] += [broker1, broker2]
                        result['broker2'] += [broker2, broker1]

                        if status1 < status2:
                            result['isFast'] += [1, 0]
                        elif status1 > status2:
                            result['isFast'] += [0, 1]
                        else:
                            result['isFast'] += [-1, -1]                

result = pd.DataFrame(result)
result = result[result['isFast'] != -1]
result['count'] = result.groupby(['broker1', 'broker2'])['isFast'].transform('count')
result['fasterSum'] = result.groupby(['broker1', 'broker2'])['isFast'].transform('sum')
result['faster'] = result['fasterSum']/result['count']


summary = pd.DataFrame()
summaryCols = []
countCols = []
fasterCols = []
for date in result['date'].unique():
    a = result[result['date'] == date].reset_index(drop=True)
    a['count'] = a.groupby(['broker1', 'broker2'])['isFast'].transform('count')
    a['fasterSum'] = a.groupby(['broker1', 'broker2'])['isFast'].transform('sum')
    a['faster'] = a['fasterSum']/a['count']
    a = a.groupby(['broker1', 'broker2'])[['count', 'faster']].mean().reset_index()
    a.columns = ['broker1', 'broker2', '%s count'%date, '%s faster'%date]
    summaryCols += ['%s count'%date, '%s faster'%date]
    countCols += ['%s count'%date]
    fasterCols += ['%s faster'%date]

    if summary.empty:
        summary = a.copy()
    else:
        summary = pd.merge(summary, a, how='outer', on=['broker1', 'broker2'], validate='one_to_one')

a = result.groupby(['broker1', 'broker2'])[['count', 'faster']].mean().reset_index()
a.columns = ['broker1', 'broker2', 'total count', 'total faster']
summary = pd.merge(summary, a, how='outer', on=['broker1', 'broker2'], validate='one_to_one')
summaryCols = ['total count', 'total faster'] + summaryCols

countCols += ['total count']
fasterCols += ['total faster']
# summary['b1'] = summary['broker1'].str[7:11].astype(int) // 100
# summary['b2'] = summary['broker2'].str[7:11].astype(int) // 100
#     summary = summary.groupby(['broker1', 'broker2'])[summaryCols].first()
for col in countCols:
    summary[col] = summary[col].fillna(0)
    summary[col] = summary[col].astype('int64')

for col in fasterCols:
    summary[col] = summary[col].fillna(-1)
    summary[col] = summary[col].apply(lambda x: '%.0f%%'%(x*100))
    summary[col] = np.where(summary[col] == '-100%', ' ', summary[col])

s1 = summary
# display(s1[(s1["broker1"] == 'zt_68')])

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


SZE


C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


SZE


C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


...

SZE


C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


SZE


C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


SZE


In [77]:
re1 = s1[(s1['broker1'] == 'zs5213_5377') ].sort_values(by='total count', ascending=False)
re2 = s1[(s1['broker1'] == 'zs5208_5377') ].sort_values(by='total count', ascending=False)
re1['accCode'] = 5377
re2['accCode'] = 5377
re1['tag'] = 'after'
re2['tag'] = 'before'
re = pd.concat([re1, re2])
re['num'] = re.groupby('broker2')['broker1'].transform('nunique')
re = re[re['num'] == 2].sort_values(by='broker2')
l = []
for i in re.columns:
    if ((re[i].values[0] != 0) | (re[i].values[1] != 0)) & ((re[i].values[0] != ' ') | (re[i].values[1] != ' ')):
        l.append(i)
re = re[l]
display(re)
re1 = re[re['tag'] == 'before'][['accCode', 'broker2', 'total count', 'total faster']]
re1 = re1.rename(columns={'total count':'before count', 'total faster':'before faster'})
re2 = re[re['tag'] == 'after'][['accCode', 'broker2', 'total count', 'total faster']]
re2 = re2.rename(columns={'total count':'after count', 'total faster':'after faster'})
re = pd.merge(re1, re2, on=['accCode', 'broker2'])
re = re[(re['before count'] > 50) & (re['after count'] > 50)]
from IPython.display import display, HTML
HTML(re.groupby(['accCode', 'broker2']).first().to_html())

,broker1,broker2,20201229 count,20201229 faster,20201230 count,20201230 faster,20201231 count,20201231 faster,20210107 count,20210107 faster,20210108 count,20210108 faster,total count,total faster,accCode,tag,num
867,zs5213_5377,zs5206_5332,0,,0,,0,,103,71%,42,50%,145,65%,5377,after,2
607,zs5208_5377,zs5206_5332,56,36%,77,36%,4,25%,0,,0,,137,36%,5377,before,2
868,zs5213_5377,zs5206_5384,0,,0,,0,,32,53%,17,71%,49,59%,5377,after,2
608,zs5208_5377,zs5206_5384,48,48%,53,19%,1,0%,0,,0,,102,32%,5377,before,2
870,zs5213_5377,zs5213_5386,0,,0,,0,,887,50%,989,57%,1876,54%,5377,after,2
710,zs5208_5377,zs5213_5386,0,,0,,29,0%,0,,0,,29,0%,5377,before,2
618,zs5208_5377,zs8804_8856,2,100%,28,11%,26,8%,0,,0,,72,14%,5377,before,2
871,zs5213_5377,zs8804_8856,0,,0,,0,,1,100%,0,,1,100%,5377,after,2
872,zs5213_5377,zs8804_8924,0,,0,,0,,294,95%,133,95%,427,95%,5377,after,2
619,zs5208_5377,zs8804_8924,34,82%,37,78%,16,56%,0,,0,,88,76%,5377,before,2


In [60]:
s1[(s1['broker1'] == 'zt5212_537401')]

,broker1,broker2,20201228 count,20201228 faster,20201229 count,20201229 faster,20201230 count,20201230 faster,20201231 count,20201231 faster,20210104 count,20210104 faster,20210105 count,20210105 faster,20210106 count,20210106 faster,20210107 count,20210107 faster,20210108 count,20210108 faster,total count,total faster
787,zt5212_537401,zs8804_892402,0,,0,,0,,0,,0,,0,,0,,16,88%,16,88%,32,88%
788,zt5212_537401,zt8806_8970,0,,0,,0,,0,,0,,0,,0,,140,46%,138,41%,278,44%
789,zt5212_537401,zt8806_8971,0,,0,,0,,0,,0,,0,,0,,74,55%,55,40%,129,49%
790,zt5212_537401,zt9406_9535,0,,0,,0,,0,,0,,0,,0,,18,94%,5,100%,23,96%
791,zt5212_537401,zt9406_9561,0,,0,,0,,0,,0,,0,,0,,24,96%,14,86%,38,92%
792,zt5212_537401,zt9609_966701,0,,0,,0,,0,,0,,0,,0,,235,20%,248,10%,483,15%


In [52]:
re['20201228 faster'].values[1]

' '

In [12]:
s1[(s1['broker1'].str[:6] == 'zs5213') & (s1['broker2'].str[:6] == 'zs5206')].sort_values(by=['broker1', 'broker2'], ascending=False)

,broker1,broker2,20201231 count,20201231 faster,20210104 count,20210104 faster,20210105 count,20210105 faster,20210106 count,20210106 faster,20210107 count,20210107 faster,20210108 count,20210108 faster,total count,total faster
272,zs5213_5386,zs5206_5384,51,55%,44,48%,0,,55,60%,30,60%,18,61%,198,56%
271,zs5213_5386,zs5206_5332,136,81%,91,67%,0,,38,55%,108,68%,32,25%,405,67%
823,zs5213_5377,zs5206_5384,0,,0,,0,,0,,32,53%,17,71%,49,59%
822,zs5213_5377,zs5206_5332,0,,0,,0,,0,,103,71%,42,50%,145,65%
786,zs5213_5329,zs5206_5384,0,,0,,0,,87,99%,201,91%,98,91%,386,92%
785,zs5213_5329,zs5206_5332,0,,0,,0,,51,90%,142,93%,53,87%,246,91%


In [66]:
orderLog[(orderLog['colo'] == 'zs_52_13') & (orderLog['updateType'] == 0)].groupby(['date', 'accCode'])['order'].nunique()

date      accCode
20201231  5386       2646
20210104  5386       2045
20210106  5329       2716
          5386       2672
Name: order, dtype: int64

In [85]:
checkLog = orderLog[(orderLog['updateType'] == 0) & (~orderLog['caa_orderLog'].isnull())].drop_duplicates(['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'], keep=False)
checkLog = checkLog.reset_index(drop=True)
checkLog['ordering'] = checkLog.index
checkLog['time_diff'] = checkLog['caa_orderLog'] - checkLog['start_time']
checkLog['colo1'] = checkLog['colo'].str[:2]
checkLog['colo_broker'] = checkLog['colo1'] + '_' + checkLog["accCode"].astype(str).str[:2]
checkLog = checkLog[checkLog['accCode'].isin([5386, 5329, 5377])]
checkLog['tag'] = 'before'
checkLog['tag'] = np.where(checkLog['colo'] == 'zs_52_13', 'after', checkLog['tag'])
ol = pd.merge(checkLog[checkLog['sta'] == '1. staone'], checkLog[checkLog['sta'] == '2. statwo'][['colo', 'accCode']].drop_duplicates(),
         on=['colo', 'accCode'], how='inner')['ordering'].unique()
ol1 = pd.merge(checkLog[checkLog['sta'] == '1. staone'], checkLog[checkLog['sta'] == '3. sta300'][['colo', 'accCode']].drop_duplicates(),
         on=['colo', 'accCode'], how='inner')['ordering'].unique()
ol2 = pd.merge(checkLog[checkLog['sta'] == '2. statwo'], checkLog[checkLog['sta'] == '3. sta300'][['colo', 'accCode']].drop_duplicates(),
         on=['colo', 'accCode'], how='inner')['ordering'].unique()
checkLog1 = checkLog[(checkLog['sta'] == '1. staone') & (~checkLog['ordering'].isin(ol)) & (~checkLog['ordering'].isin(ol1))]
checkLog2 = checkLog[(checkLog['sta'] == '2. statwo') & (~checkLog['ordering'].isin(ol2))]
checkLog3 = checkLog[checkLog['sta'] == '3. sta300']

from IPython.display import display, HTML
re1 = checkLog2.groupby(['tag', 'sta', 'colo', 'colo_broker'])['time_diff'].describe().fillna(0).astype(int).reset_index()
# re1 = re1[re1['count'] > 20].reset_index()
c1 = checkLog2.groupby(['tag', 'sta', 'colo', 'colo_broker'])['time_diff'].apply(lambda x: x.describe([0.1])['10%']).astype(int).reset_index()
c1 = c1.rename(columns={"time_diff":"10%"})
re1 = pd.merge(re1, c1[['tag', 'sta', 'colo', 'colo_broker', '10%']], on=['tag', 'sta', 'colo', 'colo_broker'])
c1 = checkLog2.groupby(['tag', 'sta', 'colo', 'colo_broker'])['time_diff'].apply(lambda x: x.describe([0.9])['90%']).astype(int).reset_index()
c1 = c1.rename(columns={"time_diff":"90%"})
re1 = pd.merge(re1, c1[['tag', 'sta', 'colo', 'colo_broker', '90%']], on=['tag', 'sta', 'colo', 'colo_broker'])
display(HTML(re1[re1['colo_broker'] == 'zs_53'][['tag', 'sta', 'colo', 'colo_broker', 'count',  '10%', '25%', '50%', '75%', '90%']].groupby(['tag', 'sta', 'colo', 'colo_broker']).first().sort_values(by='50%').to_html()))


from IPython.display import display, HTML
re3 = checkLog3.groupby(['tag', 'sta', 'colo', 'colo_broker'])['time_diff'].describe().fillna(0).astype(int).reset_index()
# re1 = re1[re1['count'] > 20].reset_index()
c1 = checkLog3.groupby(['tag', 'sta', 'colo', 'colo_broker'])['time_diff'].apply(lambda x: x.describe([0.1])['10%']).astype(int).reset_index()
c1 = c1.rename(columns={"time_diff":"10%"})
re3 = pd.merge(re3, c1[['tag', 'sta', 'colo', 'colo_broker', '10%']], on=['tag', 'sta', 'colo', 'colo_broker'])
c1 = checkLog3.groupby(['tag', 'sta', 'colo', 'colo_broker'])['time_diff'].apply(lambda x: x.describe([0.9])['90%']).astype(int).reset_index()
c1 = c1.rename(columns={"time_diff":"90%"})
re3 = pd.merge(re3, c1[['tag', 'sta', 'colo', 'colo_broker', '90%']], on=['tag', 'sta', 'colo', 'colo_broker'])
display(HTML(re3[re3['colo_broker'] == 'zs_53'][['tag', 'sta', 'colo', 'colo_broker', 'count',  '10%', '25%', '50%', '75%', '90%']].groupby(['tag', 'sta', 'colo', 'colo_broker']).first().sort_values(by='50%').to_html()))

re = pd.concat([re1, re3])
display(HTML(re[re['colo_broker'] == 'zs_53'][['tag', 'sta', 'colo', 'colo_broker', 'count',  '10%', '25%', '50%', '75%', '90%']].groupby(['tag', 'sta', 'colo', 'colo_broker']).first().sort_values(by='50%').to_html()))

,,,,count,10%,25%,50%,75%,90%
tag,sta,colo,colo_broker,,,,,,
before,2. statwo,zs_52_08,zs_53,13596,3570,3763,3974,4628,5264


,,,,count,10%,25%,50%,75%,90%
tag,sta,colo,colo_broker,,,,,,
after,3. sta300,zs_52_13,zs_53,17596,3535,3702,3891,4435,5211


,,,,count,10%,25%,50%,75%,90%
tag,sta,colo,colo_broker,,,,,,
after,3. sta300,zs_52_13,zs_53,17596,3535,3702,3891,4435,5211
before,2. statwo,zs_52_08,zs_53,13596,3570,3763,3974,4628,5264


In [89]:
orderLog[orderLog['accCode'].isin([537301, 537403, 537401])].groupby(['date', 'colo', 'sta'])['order'].nunique()

date      colo      sta      
20201228  zt_52_04  1. staone     175
                    2. statwo     526
          zt_52_05  3. sta300     783
          zt_52_07  1. staone      18
                    2. statwo    1013
20201229  zt_52_04  1. staone      26
                    2. statwo     277
          zt_52_05  2. statwo       2
                    3. sta300     795
          zt_52_07  1. staone      33
                    2. statwo     926
20201230  zt_52_04  1. staone     155
                    2. statwo     669
          zt_52_05  2. statwo       5
                    3. sta300     630
          zt_52_07  1. staone      40
                    2. statwo    1000
20201231  zt_52_05  1. staone       1
                    2. statwo      17
                    3. sta300    1181
          zt_52_07  1. staone      37
                    2. statwo    1119
          zt_52_12  1. staone      58
                    2. statwo     119
                    3. sta300    1464
20210104  zt_52_05  

In [90]:
orderLog[orderLog['accCode'].isin([5386, 5329, 5377])].groupby(['date', 'colo', 'sta'])['order'].nunique()

date      colo      sta      
20201228  zs_52_08  1. staone      666
                    2. statwo     4807
20201229  zs_52_08  1. staone      687
                    2. statwo     6341
20201230  zs_52_08  1. staone      794
                    2. statwo     5276
20201231  zs_52_08  1. staone      460
                    2. statwo     4939
          zs_52_13  1. staone      129
                    2. statwo      223
                    3. sta300     2294
20210104  zs_52_08  1. staone      470
                    2. statwo     4203
          zs_52_13  1. staone       10
                    2. statwo        4
                    3. sta300     2031
20210106  zs_52_13  1. staone       72
                    2. statwo       14
                    3. sta300     5302
20210107  zs_52_13  1. staone       82
                    2. statwo      330
                    3. sta300    10256
20210108  zs_52_13  1. staone       46
                    2. statwo        8
                    3. sta300    1

In [24]:
checkLog = orderLog[(orderLog['updateType'] == 0) & (~orderLog['caa_orderLog'].isnull()) & (orderLog['date'] >= 20201231)].drop_duplicates(['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'], keep=False)
checkLog = checkLog.reset_index(drop=True)
checkLog['ordering'] = checkLog.index
checkLog['time_diff'] = checkLog['caa_orderLog'] - checkLog['start_time']
checkLog['colo1'] = checkLog['colo'].str[:2]
checkLog['colo_broker'] = checkLog['colo1'] + '_' + checkLog["accCode"].astype(str).str[:2]
ol = pd.merge(checkLog[checkLog['sta'] == '1. staone'], checkLog[checkLog['sta'] == '2. statwo'][['colo', 'accCode']].drop_duplicates(),
         on=['colo', 'accCode'], how='inner')['ordering'].unique()
ol1 = pd.merge(checkLog[checkLog['sta'] == '1. staone'], checkLog[checkLog['sta'] == '3. sta300'][['colo', 'accCode']].drop_duplicates(),
         on=['colo', 'accCode'], how='inner')['ordering'].unique()
ol2 = pd.merge(checkLog[checkLog['sta'] == '2. statwo'], checkLog[checkLog['sta'] == '3. sta300'][['colo', 'accCode']].drop_duplicates(),
         on=['colo', 'accCode'], how='inner')['ordering'].unique()
checkLog1 = checkLog[(checkLog['sta'] == '1. staone') & (~checkLog['ordering'].isin(ol)) & (~checkLog['ordering'].isin(ol1))]
checkLog2 = checkLog[(checkLog['sta'] == '2. statwo') & (~checkLog['ordering'].isin(ol2))]
checkLog3 = checkLog[checkLog['sta'] == '3. sta300']

from IPython.display import display, HTML
re1 = checkLog1.groupby(['sta', 'colo', 'colo_broker'])['time_diff'].describe().fillna(0).astype(int).reset_index()
# re1 = re1[re1['count'] > 20].reset_index()
c1 = checkLog1.groupby(['sta', 'colo', 'colo_broker'])['time_diff'].apply(lambda x: x.describe([0.1])['10%']).astype(int).reset_index()
c1 = c1.rename(columns={"time_diff":"10%"})
re1 = pd.merge(re1, c1[['sta', 'colo', 'colo_broker', '10%']], on=['sta', 'colo', 'colo_broker'])
c1 = checkLog1.groupby(['sta', 'colo', 'colo_broker'])['time_diff'].apply(lambda x: x.describe([0.9])['90%']).astype(int).reset_index()
c1 = c1.rename(columns={"time_diff":"90%"})
re1 = pd.merge(re1, c1[['sta', 'colo', 'colo_broker', '90%']], on=['sta', 'colo', 'colo_broker'])

display(HTML(re1[re1['colo_broker'] == 'zs_53'][['sta', 'colo', 'colo_broker', 'count',  '10%', '25%', '50%', '75%', '90%']].groupby(['sta', 'colo', 'colo_broker']).first().sort_values(by='50%').to_html()))

from IPython.display import display, HTML
re1 = checkLog2.groupby(['sta', 'colo', 'colo_broker'])['time_diff'].describe().fillna(0).astype(int).reset_index()
# re1 = re1[re1['count'] > 20].reset_index()
c1 = checkLog2.groupby(['sta', 'colo', 'colo_broker'])['time_diff'].apply(lambda x: x.describe([0.1])['10%']).astype(int).reset_index()
c1 = c1.rename(columns={"time_diff":"10%"})
re1 = pd.merge(re1, c1[['sta', 'colo', 'colo_broker', '10%']], on=['sta', 'colo', 'colo_broker'])
c1 = checkLog2.groupby(['sta', 'colo', 'colo_broker'])['time_diff'].apply(lambda x: x.describe([0.9])['90%']).astype(int).reset_index()
c1 = c1.rename(columns={"time_diff":"90%"})
re1 = pd.merge(re1, c1[['sta', 'colo', 'colo_broker', '90%']], on=['sta', 'colo', 'colo_broker'])

display(HTML(re1[re1['colo_broker'] == 'zs_53'][['sta', 'colo', 'colo_broker', 'count',  '10%', '25%', '50%', '75%', '90%']].groupby(['sta', 'colo', 'colo_broker']).first().sort_values(by='50%').to_html()))

from IPython.display import display, HTML
re1 = checkLog3.groupby(['sta', 'colo', 'colo_broker'])['time_diff'].describe().fillna(0).astype(int).reset_index()
# re1 = re1[re1['count'] > 20].reset_index()
c1 = checkLog3.groupby(['sta', 'colo', 'colo_broker'])['time_diff'].apply(lambda x: x.describe([0.1])['10%']).astype(int).reset_index()
c1 = c1.rename(columns={"time_diff":"10%"})
re1 = pd.merge(re1, c1[['sta', 'colo', 'colo_broker', '10%']], on=['sta', 'colo', 'colo_broker'])
c1 = checkLog3.groupby(['sta', 'colo', 'colo_broker'])['time_diff'].apply(lambda x: x.describe([0.9])['90%']).astype(int).reset_index()
c1 = c1.rename(columns={"time_diff":"90%"})
re1 = pd.merge(re1, c1[['sta', 'colo', 'colo_broker', '90%']], on=['sta', 'colo', 'colo_broker'])

display(HTML(re1[re1['colo_broker'] == 'zs_53'][['sta', 'colo', 'colo_broker', 'count',  '10%', '25%', '50%', '75%', '90%']].groupby(['sta', 'colo', 'colo_broker']).first().sort_values(by='50%').to_html()))


,,,count,10%,25%,50%,75%,90%
sta,colo,colo_broker,,,,,,


In [26]:
checkLog = orderLog[(orderLog["updateType"] == 0) & (orderLog['date'] >= 20201231)]
checkLog = checkLog[checkLog['caamd'] != 0]
checkLog['internal_latency'] = checkLog["clockAtArrival"] - checkLog["caamd"]
SSE = checkLog[checkLog['secid'] < 2000000]
SSE["exchange"] = "SH"
SSE = SSE[(SSE['accCode'].astype(str).str[:2] == '53') & (SSE['colo'].str[:2] == 'zt')]

c1 = SSE.groupby(['colo', 'exchange', "date"])["internal_latency"].quantile(.95).reset_index().groupby(['colo', 'exchange'])["internal_latency"].mean().reset_index()
c2 = SSE.groupby(['colo', 'exchange', "date"])["internal_latency"].median().reset_index().groupby(['colo', 'exchange'])["internal_latency"].mean().reset_index()
c3 = SSE.groupby(['colo', 'exchange'])["internal_latency"].count().reset_index()
c4 = SSE.groupby(['colo', 'exchange'])["date"].unique().str.len().reset_index()
c5 = SSE.groupby(['colo', 'exchange', "date"])["internal_latency"].quantile(.95).reset_index().groupby(['colo', 'exchange'])["internal_latency"].std().reset_index()

re1 = pd.merge(c3, c1, on=['colo', 'exchange'])
re1 = re1.rename(columns = {'internal_latency_x': 'count', 'internal_latency_y': '95 percentile'})
re1 = pd.merge(re1, c2, on=['colo', 'exchange'])
re1 = re1.rename(columns = {'internal_latency': 'median'})
re1 = pd.merge(re1, c4, on=['colo', 'exchange'])
re1 = pd.merge(re1, c5, on=['colo', 'exchange'])
re1 = re1.rename(columns = {'internal_latency': 'std'})


# re1 = pd.merge(re1[re1["isMsg"] == 1], re1[re1["isMsg"] == 0], on=["exchange", "colo_account"], how="outer").sort_values(by="median_x").reset_index(drop=True)
# re2 = pd.merge(re2[re2["isMsg"] == 1], re2[re2["isMsg"] == 0], on=["exchange", "colo_account"], how="outer").sort_values(by="median_y").reset_index(drop=True)
re = pd.concat([re1]).reset_index(drop=True)


for col in ['median', '95 percentile']:
    re[col] = re[col].astype(int)
for col in ['std']:
    re[col] = re[col].apply(lambda x: '%.2f'%(x))
    
    
from IPython.display import display, HTML

display(HTML(re.groupby(['colo', 'exchange'])["count", "median", "95 percentile", "std"].first().to_html()))

SZE = checkLog[checkLog['secid'] >= 2000000]
SZE["exchange"] = "SZ"
SZE = SZE[(SZE['accCode'].astype(str).str[:2] == '53') & (SZE['colo'].str[:2] == 'zs')]

c1 = SZE.groupby(['colo', 'exchange', "date"])["internal_latency"].quantile(.95).reset_index().groupby(['colo', 'exchange'])["internal_latency"].mean().reset_index()
c2 = SZE.groupby(['colo', 'exchange', "date"])["internal_latency"].median().reset_index().groupby(['colo', 'exchange'])["internal_latency"].mean().reset_index()
c3 = SZE.groupby(['colo', 'exchange'])["internal_latency"].count().reset_index()
c4 = SZE.groupby(['colo', 'exchange'])["date"].unique().str.len().reset_index()
c5 = SZE.groupby(['colo', 'exchange', "date"])["internal_latency"].quantile(.95).reset_index().groupby(['colo', 'exchange'])["internal_latency"].std().reset_index()

re1 = pd.merge(c3, c1, on=['colo', 'exchange'])
re1 = re1.rename(columns = {'internal_latency_x': 'count', 'internal_latency_y': '95 percentile'})
re1 = pd.merge(re1, c2, on=['colo', 'exchange'])
re1 = re1.rename(columns = {'internal_latency': 'median'})
re1 = pd.merge(re1, c4, on=['colo', 'exchange'])
re1 = pd.merge(re1, c5, on=['colo', 'exchange'])
re1 = re1.rename(columns = {'internal_latency': 'std'})


# re1 = pd.merge(re1[re1["isMsg"] == 1], re1[re1["isMsg"] == 0], on=["exchange", "colo_account"], how="outer").sort_values(by="median_x").reset_index(drop=True)
# re2 = pd.merge(re2[re2["isMsg"] == 1], re2[re2["isMsg"] == 0], on=["exchange", "colo_account"], how="outer").sort_values(by="median_y").reset_index(drop=True)
re = pd.concat([re1]).reset_index(drop=True)


for col in ['median', '95 percentile']:
    re[col] = re[col].astype(int)
for col in ['std']:
    re[col] = re[col].apply(lambda x: '%.2f'%(x))
    
    
from IPython.display import display, HTML

display(HTML(re.groupby(['colo', 'exchange'])["count", "median", "95 percentile", "std"].first().to_html()))

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


,,count,median,95 percentile,std
colo,exchange,,,,
zt_52_05,SH,8696,141,432,41.95
zt_52_07,SH,2422,99,191,2.30
zt_52_12,SH,18607,75,191,6.45


C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:70: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


,,count,median,95 percentile,std
colo,exchange,,,,
zs_52_06,SZ,27768,71,150,13.09
zs_52_08,SZ,9375,59,118,0.14
zs_52_09,SZ,33024,69,175,9.23
zs_52_13,SZ,28674,61,85,6.22


In [96]:
checkLog = orderLog[(orderLog["updateType"] == 0)]
checkLog = checkLog[checkLog['caamd'] != 0]
checkLog['internal_latency'] = checkLog["clockAtArrival"] - checkLog["caamd"]
SSE = checkLog[checkLog['secid'] < 2000000]
SSE["exchange"] = "SH"
SSE = SSE[SSE['accCode'].isin([537301, 537403, 537401])]
SSE['tag'] = 'before'
SSE['tag'] = np.where(SSE['colo'] == 'zt_52_12', 'after', SSE['tag'])

c1 = SSE.groupby(['tag', 'exchange', "date"])["internal_latency"].quantile(.95).reset_index().groupby(['tag', 'exchange'])["internal_latency"].mean().reset_index()
c2 = SSE.groupby(['tag', 'exchange', "date"])["internal_latency"].median().reset_index().groupby(['tag', 'exchange'])["internal_latency"].mean().reset_index()
c3 = SSE.groupby(['tag', 'exchange'])["internal_latency"].count().reset_index()
c4 = SSE.groupby(['tag', 'exchange'])["date"].unique().str.len().reset_index()
c5 = SSE.groupby(['tag', 'exchange', "date"])["internal_latency"].quantile(.95).reset_index().groupby(['tag', 'exchange'])["internal_latency"].std().reset_index()

re1 = pd.merge(c3, c1, on=['tag', 'exchange'])
re1 = re1.rename(columns = {'internal_latency_x': 'count', 'internal_latency_y': '95 percentile'})
re1 = pd.merge(re1, c2, on=['tag', 'exchange'])
re1 = re1.rename(columns = {'internal_latency': 'median'})
re1 = pd.merge(re1, c4, on=['tag', 'exchange'])
re1 = pd.merge(re1, c5, on=['tag', 'exchange'])
re1 = re1.rename(columns = {'internal_latency': 'std'})


# re1 = pd.merge(re1[re1["isMsg"] == 1], re1[re1["isMsg"] == 0], on=["exchange", "tag_account"], how="outer").sort_values(by="median_x").reset_index(drop=True)
# re2 = pd.merge(re2[re2["isMsg"] == 1], re2[re2["isMsg"] == 0], on=["exchange", "tag_account"], how="outer").sort_values(by="median_y").reset_index(drop=True)
re = pd.concat([re1]).reset_index(drop=True)


for col in ['median', '95 percentile']:
    re[col] = re[col].astype(int)
for col in ['std']:
    re[col] = re[col].apply(lambda x: '%.2f'%(x))
    
    
from IPython.display import display, HTML

display(HTML(re.groupby(['tag', 'exchange'])["count", "median", "95 percentile", "std"].first().to_html()))

SZE = checkLog[checkLog['secid'] >= 2000000]
SZE["exchange"] = "SZ"
SZE = SZE[SZE['accCode'].isin([5386, 5329, 5377])]
SZE['tag'] = 'before'
SZE['tag'] = np.where(SZE['colo'] == 'zs_52_13', 'after', SZE['tag'])

c1 = SZE.groupby(['tag', 'exchange', "date"])["internal_latency"].quantile(.95).reset_index().groupby(['tag', 'exchange'])["internal_latency"].mean().reset_index()
c2 = SZE.groupby(['tag', 'exchange', "date"])["internal_latency"].median().reset_index().groupby(['tag', 'exchange'])["internal_latency"].mean().reset_index()
c3 = SZE.groupby(['tag', 'exchange'])["internal_latency"].count().reset_index()
c4 = SZE.groupby(['tag', 'exchange'])["date"].unique().str.len().reset_index()
c5 = SZE.groupby(['tag', 'exchange', "date"])["internal_latency"].quantile(.95).reset_index().groupby(['tag', 'exchange'])["internal_latency"].std().reset_index()

re1 = pd.merge(c3, c1, on=['tag', 'exchange'])
re1 = re1.rename(columns = {'internal_latency_x': 'count', 'internal_latency_y': '95 percentile'})
re1 = pd.merge(re1, c2, on=['tag', 'exchange'])
re1 = re1.rename(columns = {'internal_latency': 'median'})
re1 = pd.merge(re1, c4, on=['tag', 'exchange'])
re1 = pd.merge(re1, c5, on=['tag', 'exchange'])
re1 = re1.rename(columns = {'internal_latency': 'std'})


# re1 = pd.merge(re1[re1["isMsg"] == 1], re1[re1["isMsg"] == 0], on=["exchange", "tag_account"], how="outer").sort_values(by="median_x").reset_index(drop=True)
# re2 = pd.merge(re2[re2["isMsg"] == 1], re2[re2["isMsg"] == 0], on=["exchange", "tag_account"], how="outer").sort_values(by="median_y").reset_index(drop=True)
re = pd.concat([re1]).reset_index(drop=True)


for col in ['median', '95 percentile']:
    re[col] = re[col].astype(int)
for col in ['std']:
    re[col] = re[col].apply(lambda x: '%.2f'%(x))
    
    
from IPython.display import display, HTML

display(HTML(re.groupby(['tag', 'exchange'])["count", "median", "95 percentile", "std"].first().to_html()))

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


,,count,median,95 percentile,std
tag,exchange,,,,
after,SH,18607,75,191,6.45
before,SH,11816,114,272,18.74


C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:74: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


,,count,median,95 percentile,std
tag,exchange,,,,
after,SZ,28674,61,85,6.22
before,SZ,25870,60,133,16.11


In [31]:
a = orderLog[(orderLog['updateType'] == 0) & (orderLog['exchange'] == 'SSE') & (orderLog['accCode'].astype(str).str[:2] == '53') & (orderLog['colo'].str[:2] == 'zt') & (orderLog['date'] >= 20201231)].groupby('colo')['orderNtl'].sum()
b = orderLog[(orderLog['updateType'] == 4) & (orderLog['exchange'] == 'SSE') & (orderLog['accCode'].astype(str).str[:2] == '53') & (orderLog['colo'].str[:2] == 'zt') & (orderLog['date'] >= 20201231)].groupby('colo')['tradeNtl'].sum()
re = pd.merge(a, b, on='colo')
re['fill rate'] = re['tradeNtl'] / re['orderNtl']
re['fill rate'] = re['fill rate'].apply(lambda x: '%.1f%%'%(100*x))
re

,orderNtl,tradeNtl,fill rate
colo,,,
zt_52_05,1.055092e+09,8.530395e+08,80.8%
zt_52_07,2.181363e+08,1.547350e+08,70.9%
zt_52_12,1.900667e+09,1.271515e+09,66.9%


In [98]:
orderLog['tag'] = 'before'
orderLog['tag'] = np.where(orderLog['colo'] == 'zt_52_12', 'after', orderLog['tag'])
a = orderLog[(orderLog['updateType'] == 0) & (orderLog['exchange'] == 'SSE') & (orderLog['accCode'].isin([537301, 537403, 537401]))].groupby(['accCode', 'tag'])['orderNtl'].sum()
b = orderLog[(orderLog['updateType'] == 4) & (orderLog['exchange'] == 'SSE') & (orderLog['accCode'].isin([537301, 537403, 537401]))].groupby(['accCode', 'tag'])['tradeNtl'].sum()
re = pd.merge(a, b, on=['accCode', 'tag'])
re['fill rate'] = re['tradeNtl'] / re['orderNtl']
re['fill rate'] = re['fill rate'].apply(lambda x: '%.1f%%'%(100*x))
re

orderNtl      tradeNtl fill rate
accCode tag                                         
537301  after   1.369597e+09  9.231318e+08     67.4%
        before  1.849564e+08  1.074547e+08     58.1%
537401  after   2.321546e+08  1.615641e+08     69.6%
        before  4.654677e+08  3.322957e+08     71.4%
537403  after   2.989147e+08  1.868187e+08     62.5%
        before  2.757296e+08  1.747346e+08     63.4%

In [99]:
orderLog['tag'] = 'before'
orderLog['tag'] = np.where(orderLog['colo'] == 'zs_52_13', 'after', orderLog['tag'])
a = orderLog[(orderLog['updateType'] == 0) & (orderLog['exchange'] == 'SZE') & (orderLog['accCode'].isin([5386, 5329, 5377]))].groupby(['accCode', 'tag'])['orderNtl'].sum()
b = orderLog[(orderLog['updateType'] == 4) & (orderLog['exchange'] == 'SZE') & (orderLog['accCode'].isin([5386, 5329, 5377]))].groupby(['accCode', 'tag'])['tradeNtl'].sum()
re = pd.merge(a, b, on=['accCode', 'tag'])
re['fill rate'] = re['tradeNtl'] / re['orderNtl']
re['fill rate'] = re['fill rate'].apply(lambda x: '%.1f%%'%(100*x))
re

orderNtl      tradeNtl fill rate
accCode tag                                         
5329    after   5.850021e+08  4.182381e+08     71.5%
        before  5.239084e+08  3.418336e+08     65.2%
5377    after   5.338275e+08  3.687747e+08     69.1%
        before  6.445769e+08  3.930066e+08     61.0%
5386    after   8.607866e+08  5.805610e+08     67.4%
        before  2.576037e+08  1.409947e+08     54.7%

In [93]:
a = orderLog[(orderLog['updateType'] == 0) & (orderLog['exchange'] == 'SZE') & (orderLog['accCode'].astype(str).str[:2] == '53') & (orderLog['colo'].str[:2] == 'zs')].groupby(['date', 'sta', 'server_account'])['orderNtl'].sum()
b = orderLog[(orderLog['updateType'] == 4) & (orderLog['exchange'] == 'SZE') & (orderLog['accCode'].astype(str).str[:2] == '53') & (orderLog['colo'].str[:2] == 'zs')].groupby(['date', 'sta', 'server_account'])['tradeNtl'].sum()
re = pd.merge(a, b, on=['date', 'sta', 'server_account'])
re['fill rate'] = re['tradeNtl'] / re['orderNtl']
re['fill rate'] = re['fill rate'].apply(lambda x: '%.1f%%'%(100*x))
re

orderNtl      tradeNtl fill rate
date     sta       server_account                                      
20201228 1. staone zs5206_5332     8.758764e+06  6.308616e+06     72.0%
                   zs5206_5384     4.993410e+06  3.467559e+06     69.4%
                   zs5208_5329     8.410891e+06  6.501995e+06     77.3%
                   zs5208_5386     4.828989e+06  3.291621e+06     68.2%
                   zs5209_5326     1.123504e+07  6.244183e+06     55.6%
         2. statwo zs5206_5332     4.610930e+07  2.357462e+07     51.1%
                   zs5206_5384     1.308054e+07  1.028459e+07     78.6%
                   zs5208_5329     1.069753e+08  7.026123e+07     65.7%
                   zs5208_5386     1.038348e+08  5.825576e+07     56.1%
                   zs5209_5326     2.250012e+08  1.194886e+08     53.1%
         3. sta300 zs5206_5384     2.930507e+08  2.783324e+08     95.0%
20201229 1. staone zs5206_5332     7.306889e+06  3.557028e+06     48.7%
                   zs5206_5384     4.270529e+06  2.756877e+06     64.6%
                   zs5208_5329     3.529219e+06  2.646387e+06     75.0%
                   zs5208_5377     4.945252e+06  3.276421e+06     66.3%
                   zs5208_5386     2.389920e+06  1.018350e+06     42.6%
                   zs5209_5326     6.663652e+06  3.414758e+06     51.2%
         2. statwo zs5206_5332     4.494323e+07  2.226060e+07     49.5%
                   zs5206_5384     1.171816e+07  9.324266e+06     79.6%
                   zs5208_5329     8.252702e+07  5.115996e+07     62.0%
                   zs5208_5377     1.518347e+08  9.613416e+07     63.3%
                   zs5208_5386     7.814396e+07  4.194955e+07     53.7%
                   zs5209_5326     1.439010e+08  8.026040e+07     55.8%
         3. sta300 zs5206_5384     2.742808e+08  2.553745e+08     93.1%
20201230 1. staone zs5206_5332     8.210508e+06  5.398662e+06     65.8%
                   zs5206_5384     4.451862e+06  3.354287e+06     75.3%
                   zs5208_5329     5.490400e+06  3.341734e+06     60.9%
                   zs5208_5377     8.766412e+06  5.065826e+06     57.8%
                   zs5208_5386     3.928782e+06  2.244434e+06     57.1%
                   zs5209_5326     1.989266e+07  1.473704e+07     74.1%
         2. statwo zs5206_5332     2.723263e+07  1.246875e+07     45.8%
                   zs5206_5384     9.766124e+06  6.841622e+06     70.1%
                   zs5208_5329     6.803499e+07  4.384808e+07     64.4%
                   zs5208_5377     1.110130e+08  6.916758e+07     62.3%
                   zs5208_5386     6.447726e+07  3.423502e+07     53.1%
                   zs5209_5326     1.539021e+08  9.996285e+07     65.0%
         3. sta300 zs5206_5384     2.785547e+08  2.565900e+08     92.1%
20201231 1. staone zs5206_5332     3.421332e+06  3.000177e+06     87.7%
                   zs5208_5329     7.956895e+06  4.866545e+06     61.2%
                   zs5208_5377     1.069275e+07  5.647126e+06     52.8%
                   zs5209_5326     7.335837e+06  4.554796e+06     62.1%
                   zs5213_5386     8.181900e+05  6.646690e+05     81.2%
         2. statwo zs5206_5332     7.991819e+06  5.736013e+06     71.8%
                   zs5206_5384     2.310515e+06  2.096748e+06     90.7%
                   zs5208_5329     1.100594e+08  7.065974e+07     64.2%
                   zs5208_5377     1.772779e+08  1.109054e+08     62.6%
                   zs5209_5326     2.048548e+08  1.212925e+08     59.2%
                   zs5213_5386     1.874519e+06  1.450814e+06     77.4%
         3. sta300 zs5206_5332     5.800309e+07  3.265056e+07     56.3%
                   zs5206_5384     3.840678e+08  3.632681e+08     94.6%
                   zs5213_5386     1.392020e+08  9.901516e+07     71.1%
20210104 1. staone zs5206_5332     6.404218e+06  5.431010e+06     84.8%
                   zs5206_5384     4.470710e+06  3.754712e+06     84.0%
                   zs5208_5329     1.423396e+07  7.354194e+06     51.7%
       

In [35]:
SSE = orderLog[(orderLog["updateType"] == 0) & (orderLog['caamd'] != 0)]
SSE['internal_latency'] = SSE["clockAtArrival"] - SSE["caamd"]
SSE = SSE[SSE['secid'] < 2000000]
SSE["exchange"] = "SH"
SSE = SSE[(SSE['accCode'].astype(str).str[:2] == '53') & (SSE['colo'].str[:2] == 'zt')]
SSE['tag'] = '1228-1230'
SSE['tag'] = np.where(SSE['date'] >= 20201231, '1231-0108', SSE['tag'])

SSE['nn'] = SSE.groupby(['exchange', 'secid'])['tag'].transform('nunique')
SSE = SSE[SSE['nn'] == 2]
SSE['broker'] = SSE['colo'].str[:2] + '_' + SSE['accCode'].astype(str).str[:2]
re = SSE.groupby(['tag', 'broker'])['internal_latency'].describe().reset_index()
re = re[['tag', 'broker', 'count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']]
re = re.fillna(0)
for i in re.columns[(re.columns != 'tag') & (re.columns != 'broker')]:
    re[i] = re[i].astype('int64')
    
from IPython.display import display, HTML
HTML(re.groupby(['broker', 'tag'])[['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']].first().to_html())

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [36]:
SZE = orderLog[(orderLog["updateType"] == 0) & (orderLog['caamd'] != 0)]
SZE['internal_latency'] = SZE["clockAtArrival"] - SZE["caamd"]
SZE = SZE[SZE['secid'] > 2000000]
SZE["exchange"] = "SZ"
SZE = SZE[(SZE['accCode'].astype(str).str[:2] == '53') & (SZE['colo'].str[:2] == 'zs')]
SZE['tag'] = '1228-1230'
SZE['tag'] = np.where(SZE['date'] >= 20201231, '1231-0108', SZE['tag'])

SZE['nn'] = SZE.groupby(['exchange', 'secid'])['tag'].transform('nunique')
SZE = SZE[SZE['nn'] == 2]
SZE['broker'] = SZE['colo'].str[:2] + '_' + SZE['accCode'].astype(str).str[:2]
re = SZE.groupby(['tag', 'broker'])['internal_latency'].describe().reset_index()
re = re[['tag', 'broker', 'count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']]
re = re.fillna(0)
for i in re.columns[(re.columns != 'tag') & (re.columns != 'broker')]:
    re[i] = re[i].astype('int64')
    
from IPython.display import display, HTML
HTML(re.groupby(['broker', 'tag'])[['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']].first().to_html())

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [37]:
checkLog1 = orderLog[(orderLog['accCode'].astype(str).str[:2] == '53') & (orderLog['exchange'] == 'SZE') & (orderLog['colo'].str[:2] == 'zs')]
checkLog2 = orderLog[(orderLog['accCode'].astype(str).str[:2] == '53') & (orderLog['exchange'] == 'SSE') & (orderLog['colo'].str[:2] == 'zt')]
checkLog = pd.concat([checkLog1, checkLog2])
checkLog['tag'] = '1228-1230'
checkLog['tag'] = np.where(checkLog['date'] >= 20201231, '1231-0108', checkLog['tag'])
checkLog['count'] = checkLog.groupby(['exchange', 'secid'])['tag'].transform('nunique')
checkLog = checkLog[checkLog['count'] == 2]

d1 = checkLog[(checkLog['updateType'] == 0)].groupby(['broker', 'tag', 'exchange'])['orderNtl'].sum().reset_index()
d2 = checkLog[(checkLog['updateType'] == 4)].groupby(['broker', 'tag', 'exchange'])['tradeNtl'].sum().reset_index()
re2 = pd.merge(d1, d2, on=['broker', 'tag', 'exchange'])
re2['fill rate'] = re2['tradeNtl'] / re2['orderNtl']
re2['fill rate'] = re2['fill rate'].apply(lambda x: '%.1f%%'%(100*x))
from IPython.display import display, HTML
HTML(re2.groupby(['exchange','broker', 'tag'])['orderNtl', 'tradeNtl', 'fill rate'].first().to_html())

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  from ipykernel import kernelapp as app


In [44]:
checkLog = orderLog[(orderLog['updateType'] == 0) & (~orderLog['caa_orderLog'].isnull())].drop_duplicates(['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'], keep=False)
checkLog = checkLog.reset_index(drop=True)
checkLog['ordering'] = checkLog.index
checkLog['time_diff'] = checkLog['caa_orderLog'] - checkLog['start_time']
checkLog['colo1'] = checkLog['colo'].str[:2]
checkLog['colo_broker'] = checkLog['colo1'] + '_' + checkLog["accCode"].astype(str).str[:2]
checkLog = checkLog[checkLog['sta'] == '3. sta300']
checkLog['tag'] = '1228-1230'
checkLog['tag'] = np.where(checkLog['date'] >= 20201231, '1231-0108', checkLog['tag'])
checkLog['count'] = checkLog.groupby(['exchange', 'secid'])['tag'].transform('nunique')
checkLog = checkLog[checkLog['count'] == 2]

re1 = checkLog.groupby(['sta', 'tag', 'colo_broker'])['time_diff'].describe().fillna(0).astype(int).reset_index()
# re1 = re1[re1['count'] > 20].reset_index()
c1 = checkLog.groupby(['sta', 'tag', 'colo_broker'])['time_diff'].apply(lambda x: x.describe([0.1])['10%']).astype(int).reset_index()
c1 = c1.rename(columns={"time_diff":"10%"})
re1 = pd.merge(re1, c1[['sta', 'tag', 'colo_broker', '10%']], on=['sta', 'tag', 'colo_broker'])
c1 = checkLog.groupby(['sta', 'tag', 'colo_broker'])['time_diff'].apply(lambda x: x.describe([0.9])['90%']).astype(int).reset_index()
c1 = c1.rename(columns={"time_diff":"90%"})
re1 = pd.merge(re1, c1[['sta', 'tag', 'colo_broker', '90%']], on=['sta', 'tag', 'colo_broker'])

display(HTML(re1[re1['colo_broker'] == 'zs_53'][['sta', 'tag', 'colo_broker', 'count',  '10%', '25%', '50%', '75%', '90%']].groupby(['sta', 'tag', 'colo_broker']).first().to_html()))